## Agents

In [50]:
# Run this cell if you want to make your display wider
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

- The core idea of agents is to use a language model to choose a sequence of actions to take.
- In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

### Tools

A 'capability' of an agent.

#### Toolkit
Groups of tools that your agent can select from

## AgentAction
This is a dataclass that represents the action an agent should take. It has a tool property (which is the name of the tool that should be invoked) and a tool_input property (the input to that tool)

## AgentFinish
This represents the final result from an agent, when it is ready to return to the user. It contains a return_values key-value mapping, which contains the final agent output. Usually, this contains an output key containing a string that is the agent's response.

## Intermediate Steps
These represent previous agent actions and corresponding outputs from this CURRENT agent run. These are important to pass to future iteration so the agent knows what work it has already done. This is typed as a List[Tuple[AgentAction, Any]]. Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string.

## Agent
This is the chain responsible for deciding what step to take next. This is usually powered by a language model, a prompt, and an output parser.

### Agent Inputs
The inputs to an agent are a key-value mapping. There is only one required key: intermediate_steps, which corresponds to Intermediate Steps as described above.

## Agent Outputs
The output is the next action(s) to take or the final response to send to the user (AgentActions or AgentFinish). Concretely, this can be typed as Union[AgentAction, List[AgentAction], AgentFinish].

## AgentExecutor
The agent executor is the runtime for an agent. This is what actually calls the agent, executes the actions it chooses, passes the action outputs back to the agent, and repeats.


next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action



Toolkit - group of tools required to accomplish some task. The took is a function a agent can invoke.

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
import json
import os

llm = OpenAI(temperature=0, openai_api_key=os.getenv("GPT4_API_KEY"))

/Users/uzair/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [2]:
serp_api_key  = os.getenv("SERPAPI_API_KEY")



In [4]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=serp_api_key)


In [5]:
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)


/Users/uzair/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
response = agent({"input":"what was the first album of the" 
                    "band that Natalie Bergman is a part of?"})

/Users/uzair/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should search for the name of the band and their first album
Action: Search
Action Input: "Natalie Bergman band first album"
Observation: Wild Belle is an American band, composed of siblings Elliot Bergman and Natalie Bergman. They appeared on Conan on November 26, 2012. Their debut album, Isles, was released by Columbia Records on March 12, 2013.
Thought: Now I know the name of the band and their first album
Action: Search
Action Input: "Wild Belle Isles"
Observation: ['Isles is the debut studio album by the American brother/sister duo Wild Belle. It was released on March 12, 2013, by Columbia Records. The album encompasses many genres, and its title refers to the band\'s goal of making each song its own "isle" with a distinct musical style.', 'Isles type: Studio album by Wild Belle.', 'Isles main_tab_text: Overview.', 'Isles kgmid: /m/0rpgkdd.', 'Isles artist: Wild Belle.', 'Isles genres: Pop music, Electronica.', 'Isles release_date: March 

In [7]:
response = agent({"input":"What is ukrain and russia conflict?"})



> Entering new AgentExecutor chain...
 I should search for news articles and reliable sources to understand the current situation.
Action: Search
Action Input: "Ukraine and Russia conflict"
Observation: [{'title': 'Russia-Ukraine war at a glance: what we know on day 733', 'link': 'https://www.theguardian.com/world/2024/feb/26/russia-ukraine-war-at-a-glance-what-we-know-on-day-733', 'source': 'The Guardian', 'date': '3 hours ago', 'thumbnail': 'https://serpapi.com/searches/65dc22d65b54ef48ca18c504/images/e9bbf06ac16fb457fe76371e35c715dfc53041a8b9644b30.jpeg'}, {'title': 'February 25, 2024 - Russia-Ukraine news', 'link': 'https://www.cnn.com/europe/live-news/russia-ukraine-war-news-02-25-24/index.html', 'source': 'CNN', 'date': '32 mins ago', 'thumbnail': 'https://serpapi.com/searches/65dc22d65b54ef48ca18c504/images/e9bbf06ac16fb45746f97fe0905c3f3197dac77fd1709191.jpeg', 'live': True}, {'title': '31000 Ukrainian Soldiers Killed in Two Years of War, Zelensky Says', 'link': 'https://www.

# Different Use Cases of LangChain

- Summarization - Express the most important facts about a body of text or chat interaction
- Question and Answering Over Documents - Use information held within documents to answer questions or query
- Extraction - Pull structured data from a body of text or an user query
- Evaluation - Understand the quality of output from your application
- Querying Tabular Data - Pull data from databases or other tabular source
- Code Understanding - Reason about and digest code
- Interacting with APIs - Query APIs and interact with the outside world
- Chatbots - A framework to have a back and forth interaction with a user combined with memory in a chat interface
- Agents - Use LLMs to make decisions about what to do next. Enable these decisions with tools.

## Summaries Of Longer Text


In [13]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [15]:
with open('./the_spring.txt', 'r') as file:
    text = file.read()

In [16]:
num_tokens = llm.get_num_tokens(text)

print (f"There are {num_tokens} tokens in your file")

There are 8527 tokens in your file


In [27]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n"], chunk_size=2000, chunk_overlap=500)
docs = text_splitter.create_documents([text])

print (f"You now have {len(docs)} docs intead of 1 piece of text")

You now have 7 docs intead of 1 piece of text


In [28]:
chain = load_summarize_chain(llm=llm, chain_type='map_reduce',verbose=True ) # verbose=True optional to see what is getting sent to the LLM


In [29]:
output = chain.run(docs)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"The air of Kandor held the sharpness of new spring when Lan returned to the lands where he had always known he would die. Trees bore the first red of new growth, and a few scattered wildflowers dotted winter-brown grass where shadows did not cling to patches of snow, yet the pale sun offered little
warmth after the south, a gusting breeze cut through his coat, and grey clouds hinted at more than rain. He was almost home. Almost.
A hundred generations had beaten the wide road nearly as hard as the stone of the surrounding hills, and little dust rose, though a steady stream of ox-carts was leaving the morning farmers' markets in Canluum and merchant trains of tall wagons, surrounded by mounted guards in steel caps and bits of armour, flowed towards the city's high grey walls. Here and there the chains of the Kandori merchants' guild sp

## Question & Answering Using Documents As Context

In order to use LLMs for question and answer we must:

- Pass the LLM relevant context it needs to answer a question
- Pass it our question that we want answered


### Simple Q&A Example
Here let's review the convention of llm(your context + your question) = your answer

In [30]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))

In [41]:
context = """
Three drinks- Coke, Limca or Either (Either Coke or Limca) are offered by a machine at the bus stop. But the machine has been wired up wrongly by the staff at the bus stand so that each button does not give what it claims. 
"""

question = "If each drink costs Rs 10, how much minimum money do you have to put into the machine to work out which button gives which selection?"

In [42]:
output = llm(context + question)


In [44]:
output.strip()

'To work out which button gives which selection, you would need to put in a minimum of Rs 30 into the machine. This is because you would need to try all three buttons at least once to determine which one gives which drink. If you put in Rs 30, you can try each button once and still have Rs 10 left over to purchase the drink of your choice.'

## Using Embeddings for QnA


It's the process of splitting your text, embedding the chunks, putting the embeddings in a DB, and then querying them.

The goal is to select relevant chunks of our long text, but which chunks do we pull? The most popular method is to pull similar texts based off comparing vector embeddings.

In [46]:
from langchain import OpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings

In [48]:
llm = OpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))
loader = TextLoader('./the_spring.txt')
doc = loader.load()

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_splitter.split_documents(doc)

In [51]:
docs

[Document(page_content="The air of Kandor held the sharpness of new spring when Lan returned to the lands where he had always known he would die. Trees bore the first red of new growth, and a few scattered wildflowers dotted winter-brown grass where shadows did not cling to patches of snow, yet the pale sun offered little\nwarmth after the south, a gusting breeze cut through his coat, and grey clouds hinted at more than rain. He was almost home. Almost.\nA hundred generations had beaten the wide road nearly as hard as the stone of the surrounding hills, and little dust rose, though a steady stream of ox-carts was leaving the morning farmers' markets in Canluum and merchant trains of tall wagons, surrounded by mounted guards in steel caps and bits of armour, flowed towards the city's high grey walls. Here and there the chains of the Kandori merchants' guild spanned a chest or an Arafellin wore bells, a ruby decorated this man's ear, a pearl brooch that woman's breast, but for the most p

In [52]:
# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

In [53]:
num_total_characters

37875

In [58]:
embeddings = OpenAIEmbeddings(openai_api_key = os.getenv("OPENAI_API_KEY"))


In [59]:
# Embed your documents and combine with the raw text in a pseudo db. Note: This will make an API call to OpenAI
docsearch = FAISS.from_documents(docs, embeddings)

In [60]:
# retrieval engine
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [69]:
query = "who is Bukama?"
qa.run(query)

" Bukama is Lan's friend and armsman."

## Extraction

Extraction is the process of parsing data from a piece of text. This is commonly used with output parsing in order to structure our data.



In [70]:
# To help construct our Chat Messages
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# We will be using a chat model, defaults to gpt-3.5-turbo
from langchain.chat_models import ChatOpenAI

# To parse outputs and get structured data back
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

chat_model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo', openai_api_key= os.getenv("OPENAI_API_KEY"))

/Users/uzair/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [71]:
instructions = """
You will be given a sentence with fruit names, extract those fruit names and assign an emoji to them
Return the fruit name and emojis in a python dictionary
"""

fruit_names = """
Apple, Pear, this is an kiwi
"""

In [72]:
# Make your prompt which combines the instructions w/ the fruit names
prompt = (instructions + fruit_names)

# Call the LLM
output = chat_model([HumanMessage(content=prompt)])

print (output.content)
print (type(output.content))

/Users/uzair/miniconda3/envs/langchain/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{
    "Apple": "🍎",
    "Pear": "🍐",
    "kiwi": "🥝"
}
<class 'str'>


In [73]:
output_dict = eval(output.content)

print (output_dict)
print (type(output_dict))

{'Apple': '🍎', 'Pear': '🍐', 'kiwi': '🥝'}
<class 'dict'>


## Using LangChain's Response Schema
LangChain's response schema will does two things for us:

Autogenerate the a prompt with bonafide format instructions. This is great because I don't need to worry about the prompt engineering side, I'll leave that up to LangChain!

Read the output from the LLM and turn it into a proper python object for me

In [76]:
response_schemas = [
    ResponseSchema(name="artist", description="The name of the musical artist"),
    ResponseSchema(name="song", description="The name of the song that the artist plays")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)


In [77]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```


In [78]:
prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("Given a command from the user, extract the artist and song names \n \
                                                    {format_instructions}\n{user_prompt}")  
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

In [81]:
fruit_query = prompt.format_prompt(user_prompt="I really like Tum Mile by Javed")
print (fruit_query.messages[0].content)

Given a command from the user, extract the artist and song names 
                                                     The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```
I really like Tum Mile by Javed


In [82]:
fruit_output = chat_model(fruit_query.to_messages())
output = output_parser.parse(fruit_output.content)

print (output)
print (type(output))

{'artist': 'Javed', 'song': 'Tum Mile'}
<class 'dict'>


## Querying Tabular Data

The most common type of data in the world sits in tabular form (ok, ok, besides unstructured data). It is super powerful to be able to query this data with LangChain and pass it through to an LLM

In [106]:
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

db_chain.run("what are the columns in the table?")




> Entering new SQLDatabaseChain chain...
what are the columns in the table?
SQLQuery:SELECT * FROM pragma_table_info('table')
SQLResult: 
Answer:The columns in the table are "cid", "name", "type", "notnull", "dflt_value", "pk".
> Finished chain.


'The columns in the table are "cid", "name", "type", "notnull", "dflt_value", "pk".'

In [112]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [113]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    "./banking.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [114]:
agent.run("how many rows are there?")





> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)4118841188 is the number of rows in the dataframe
Final Answer: 41188

> Finished chain.


'41188'

In [115]:
agent.run("how many married people are there?")




> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the marital column is "married" and then count the number of rows.
Action: python_repl_ast
Action Input: df[df["marital"] == "married"].count()age               24928
job               24928
marital           24928
education         24928
default           24928
housing           24928
loan              24928
contact           24928
month             24928
day_of_week       24928
duration          24928
campaign          24928
pdays             24928
previous          24928
poutcome          24928
emp_var_rate      24928
cons_price_idx    24928
cons_conf_idx     24928
euribor3m         24928
nr_employed       24928
y                 24928
dtype: int6424928 is the number of married people in the dataframe.
Final Answer: 24928

> Finished chain.


'24928'

In [116]:
agent.run("whats the square root of the average age?")





> Entering new AgentExecutor chain...
Thought: I need to calculate the average age first and then take the square root.
Action: python_repl_ast
Action Input: df['age'].mean()40.0240604059434840.02406040594348 is the average age.
Action: python_repl_ast
Action Input: math.sqrt(40.02406040594348)NameError: name 'math' is not definedI need to import the math library to use the sqrt function.
Action: python_repl_ast
Action Input: import mathI have now imported the math library.
Action: python_repl_ast
Action Input: math.sqrt(40.02406040594348)6.326457176488556.32645717648855 is the square root of the average age.
Final Answer: 6.32645717648855

> Finished chain.


'6.32645717648855'

In [117]:
agent.run("whats many people are doing management job?")





> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include rows where the job is "management" and then count the number of rows.
Action: python_repl_ast
Action Input: df[df["job"] == "management"].count()age               2924
job               2924
marital           2924
education         2924
default           2924
housing           2924
loan              2924
contact           2924
month             2924
day_of_week       2924
duration          2924
campaign          2924
pdays             2924
previous          2924
poutcome          2924
emp_var_rate      2924
cons_price_idx    2924
cons_conf_idx     2924
euribor3m         2924
nr_employed       2924
y                 2924
dtype: int642924 people are doing management job.
Final Answer: 2924

> Finished chain.


'2924'

## ChatBot

Chatbots use many of the tools we've already looked at with the addition of an important topic: Memory. 


In [119]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate

# Chat specific components
from langchain.memory import ConversationBufferMemory

In [120]:
template = """
You are a chatbot that is solving the coding question in python language.

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [121]:
llm_chain = LLMChain(
    llm=OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY")), 
    prompt=prompt, 
    verbose=True, 
    memory=memory
)

In [122]:
llm_chain.predict(human_input="explain the concept of recurssion")




> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is solving the coding question in python language.


Human: explain the concept of recurssion
Chatbot:

> Finished chain.


' Recursion is a programming technique where a function calls itself repeatedly until a certain condition is met. This allows for solving complex problems by breaking them down into smaller, simpler versions of itself. The base case is the condition that stops the recursive function from calling itself, and the recursive case is where the function calls itself again with a modified input. This process continues until the base case is reached, and the function then returns the solution to the original problem. Recursion is commonly used in algorithms such as sorting and searching, and it can also be used to solve problems that can be broken down into smaller subproblems.'

In [123]:
llm_chain.predict(human_input="code in python about the concept I asked you earlier")




> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is solving the coding question in python language.

Human: explain the concept of recurssion
AI:  Recursion is a programming technique where a function calls itself repeatedly until a certain condition is met. This allows for solving complex problems by breaking them down into smaller, simpler versions of itself. The base case is the condition that stops the recursive function from calling itself, and the recursive case is where the function calls itself again with a modified input. This process continues until the base case is reached, and the function then returns the solution to the original problem. Recursion is commonly used in algorithms such as sorting and searching, and it can also be used to solve problems that can be broken down into smaller subproblems.
Human: code in python about the concept I asked you earlier
Chatbot:

> Finished chain.


' Sure, here is an example of a recursive function in Python that calculates the factorial of a number:\n\ndef factorial(n):\n    if n == 0:\n        return 1\n    else:\n        return n * factorial(n-1)\n\nThis function takes in a number and checks if it is equal to 0. If so, it returns 1 since the factorial of 0 is 1. If not, it calls itself again with the number decreased by 1 and multiplies it with the current number. This process continues until the base case is reached, and the final result is returned.'

In [124]:
llm_chain.predict(human_input="what is this concept is called in laymen language")




> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is solving the coding question in python language.

Human: explain the concept of recurssion
AI:  Recursion is a programming technique where a function calls itself repeatedly until a certain condition is met. This allows for solving complex problems by breaking them down into smaller, simpler versions of itself. The base case is the condition that stops the recursive function from calling itself, and the recursive case is where the function calls itself again with a modified input. This process continues until the base case is reached, and the function then returns the solution to the original problem. Recursion is commonly used in algorithms such as sorting and searching, and it can also be used to solve problems that can be broken down into smaller subproblems.
Human: code in python about the concept I asked you earlier
AI:  Sure, here is an example of a recursive function in Python that calculates t

" In simpler terms, recursion is a programming technique where a function solves a problem by breaking it down into smaller versions of itself until a solution is reached. It's like solving a puzzle by breaking it into smaller pieces and putting them together."